In [1]:
from facet_ml.classification import mask_rcnn
import torch
from torch import nn
import numpy as np
from torchvision.models import resnet152
from pathlib import Path
from torch.utils.data import DataLoader
import os


# train on the GPU or on the CPU, if a GPU is not available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

# # Load data
# data_folder = (
#     Path(__file__).parent.parent.parent.parent / "ProcessedData" / "Coco_v5"
# )
data_folder = r"C:\Users\Jacob\Desktop\Academics\Mirkin\colloidal_crystal_ML\ProcessedData\Coco_v5"
csv_path = r"C:\Users\Jacob\Desktop\Academics\Mirkin\colloidal_crystal_ML\ProcessedData\Training_Data_20240216\2024_02_27_Rachel-C_Processed.csv"
h5_path = r"C:\Users\Jacob\Desktop\Academics\Mirkin\colloidal_crystal_ML\ProcessedData\Training_Data_20240216\2024_02_16_Rachel-C_Training.h5"
# dataloaders, datasizes = cnn.load_colloidal_datasets_coco(
#     str(data_folder),
#     num_workers=16,
#     batch_size=2,
# )
import torchvision.transforms.v2 as T
def get_transform(train):
        transforms = []
        if train:
            transforms.append(T.RandomHorizontalFlip(0.5))
        transforms.append(T.ToDtype(torch.float, scale=True))
        transforms.append(T.ToPureTensor())
        return T.Compose(transforms)
dataset = mask_rcnn.CocoColloidalDataset(
    os.path.join(data_folder,"train"),
    os.path.join(data_folder,"train","_annotations.coco.json"),
    transforms=get_transform(True)
    )
dataloader = DataLoader(dataset, batch_size=4,shuffle=True,collate_fn=lambda x: tuple(zip(*x))) 

cuda
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [2]:
import torchvision.models.detection as detection
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import torchvision

def get_model_instance_segmentation(num_classes):
        # load an instance segmentation model pre-trained on COCO
        model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

        # get number of input features for the classifier
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        # replace the pre-trained head with a new one
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

        # now get the number of input features for the mask classifier
        in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
        hidden_layer = 256
        # and replace the mask predictor with a new one
        model.roi_heads.mask_predictor = MaskRCNNPredictor(
            in_features_mask, hidden_layer, num_classes
        )

        return model

# Load the Mask R-CNN model with a ResNet-50 backbone
model = get_model_instance_segmentation(1)



In [3]:
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# let's train it just for 2 epochs
num_epochs = 5

from engine import *

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, dataloader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    # evaluate(model, data_loader_test, device=device)

print("That's it!")

import matplotlib.pyplot as plt

from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks

RuntimeError: The size of tensor a (1024) must match the size of tensor b (3) at non-singleton dimension 0